In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import load_model
import os

from keras.optimizers import Adam

from fully_conv_model_for_lidar import fcn_model
from util_func import *
from train_2 import *

from keras.utils.generic_utils import get_custom_objects
#loss = SSD_Loss(neg_pos_ratio=neg_pos_ratio, alpha=alpha)
get_custom_objects().update({"my_loss": my_loss})

#import keras.losses
#keras.losses.custom_loss = my_loss

Using TensorFlow backend.


In [2]:
model = load_model('saved_model/model_May_27_195_99.h5')

[<tf.Tensor 'gradients/split_1_grad/concat:0' shape=(?, 64, 256, 8) dtype=float32>, None, None]


data_dir = './extract_kiti/'
list_of_view = list_of_data(data_dir)

#list_of_view = list_of_data(data_dir)
#list_of_view = [list_of_view[6831], list_of_view[7279]]

n = np.random.randint(len(list_of_view))
#n = 0
print(n)
view_file = list_of_view[n]
lidar_file =  view_file[:20] + 'lidar/lidar_' + view_file[30:] 
gtbox_file = './new_gt_boxes/all_boxes/' + view_file[15:20] + 'gt_boxes3d/gt_boxes3d_' + view_file[30:]
print(view_file)
print(lidar_file)
print(gtbox_file)

lidar = np.load(lidar_file)
gt_box = np.load(gtbox_file)
view, box = cylindrical_projection_for_training(lidar, gt_box)
print(view.shape, box.shape)

In [13]:
offset_range = 5*np.pi/180
#flip = np.random.randint(2)
flip = 0
offset = np.random.uniform(low=-offset_range, high=offset_range)
#offset = -5*np.pi/180
#offset = 0
print(offset)
lidar, gt_box = augmentation(offset, flip, lidar, gt_box)

-0.0296795099008


In [5]:
import mayavi.mlab

def viz_mayavi_with_labels(points, boxes, view_boxes = True, vals="distance"):
    x = points[:, 0]  # x position of point
    y = points[:, 1]  # y position of point
    z = points[:, 2]  # z position of pointfrom mpl_toolkits.mplot3d import Axes3D
    # r = lidar[:, 3]  # reflectance value of point
    d = np.sqrt(x ** 2 + y ** 2)  # Map Distance from sensor

    # Plot using mayavi -Much faster and smoother than matplotlib
    #import mayavi.mlab
    if vals == "height":
        col = z
    else:
        col = d

    fig = mayavi.mlab.figure(bgcolor=(0, 0, 0), size=(640, 360))
    mayavi.mlab.points3d(x, y, z,
                         col,          # Values used for Color
                         mode="point",
                         colormap='spectral', # 'bone', 'copper', 'gnuplot'
                         # color=(0, 1, 0),   # Used a fixed (r,g,b) instead
                         figure=fig,
                         )
    
    if view_boxes:
        for i in range(len(boxes)):
            car = boxes[i]
            x = car[:,0]
            y = car[:,1]
            z = car[:,2]

            mayavi.mlab.plot3d(x[:4], y[:4], z[:4], tube_radius=0.025)#, colormap='Spectral')
            mayavi.mlab.plot3d(x[[0,3]], y[[0,3]], z[[0,3]], tube_radius=0.025)
            mayavi.mlab.plot3d(x[[0,4]], y[[0,4]], z[[0,4]], tube_radius=0.025)
            mayavi.mlab.plot3d(x[[1,5]], y[[1,5]], z[[1,5]], tube_radius=0.025)
            mayavi.mlab.plot3d(x[[2,6]], y[[2,6]], z[[2,6]], tube_radius=0.025)
            mayavi.mlab.plot3d(x[[3,7]], y[[3,7]], z[[3,7]], tube_radius=0.025)


            mayavi.mlab.plot3d(x[-4:], y[-4:], z[-4:], tube_radius=0.025)#, colormap='Spectral')
            mayavi.mlab.plot3d(x[[4,7]], y[[4,7]], z[[4,7]], tube_radius=0.025)
        
    mayavi.mlab.show()

In [14]:
viz_mayavi_with_labels(lidar, gt_box)

In [15]:
all_boxes, cluster_boxes = predict_boxes(model,lidar, cluster = True,
                                         seg_thres=0.5, cluster_dist = 0.2, min_dist = 1.5, neigbor_thres = 3)
print(len(all_boxes))
print(len(cluster_boxes))

1738
5


In [16]:
n_boxes = len(all_boxes)
print(len(all_boxes))
choosen_boxes = np.random.randint(n_boxes, size = 30)
print(choosen_boxes)
viz_mayavi_with_labels(lidar, all_boxes[choosen_boxes])

1738
[ 456 1239  210  728  751  758  609 1003   11  338  596  505  472   24  162
 1513 1233  472 1047  658  321  852  871  477  184 1595 1651  328 1492  635]


In [17]:
#print(len(cluster_boxes))
viz_mayavi_with_labels(lidar, cluster_boxes)